## Deal with datetime and categorical axes

### Datetime axes
Dealing with date and time series is another common task. Bokeh has a sophisticated `DatetimeAxis` that can change the displayed ticks based on the current scale of the plot. There are some inputs for which Bokeh will automatically default to DatetimeAxis, but you can always explicitly ask for one by passing the value "datetime" to the `x_axis_type` or `y_axis_type` parameters to `figure()`. A few things of interest to look out for in this example:

- setting the width and height arguments to figure()
- customizing plots and other objects by assigning values to their attributes
- accessing guides and annotations with convenience Figure attributes: `legend, grid, xgrid, ygrid, axis, xaxis, yaxis`

In [1]:
import bokeh
# bokeh.sampledata.download() # download the sample data

Using data directory: /Users/valerio/.bokeh/data
Downloading: CGM.csv (1589982 bytes)
   1589982 [100.00%]
Downloading: US_Counties.zip (3182088 bytes)
   3182088 [100.00%]
Unpacking: US_Counties.csv
Downloading: us_cities.json (713565 bytes)
    713565 [100.00%]
Downloading: unemployment09.csv (253301 bytes)
    253301 [100.00%]
Downloading: AAPL.csv (166698 bytes)
    166698 [100.00%]
Downloading: FB.csv (9706 bytes)
      9706 [100.00%]
Downloading: GOOG.csv (113894 bytes)
    113894 [100.00%]
Downloading: IBM.csv (165625 bytes)
    165625 [100.00%]
Downloading: MSFT.csv (161614 bytes)
    161614 [100.00%]
Downloading: WPP2012_SA_DB03_POPULATION_QUINQUENNIAL.zip (5148539 bytes)
   5148539 [100.00%]
Unpacking: WPP2012_SA_DB03_POPULATION_QUINQUENNIAL.csv
Downloading: gapminder_fertility.csv (64346 bytes)
     64346 [100.00%]
Downloading: gapminder_population.csv (94509 bytes)
     94509 [100.00%]
Downloading: gapminder_life_expectancy.csv (73243 bytes)
     73243 [100.00%]
Downloading

In [2]:
from bokeh.sampledata.stocks import AAPL
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.resources import CDN
output_notebook(CDN)

Loading BokehJS ...

In [3]:
# prepare some data
aapl = np.array(AAPL['adj_close'])
aapl_dates = np.array(AAPL['date'], dtype=np.datetime64)

window_size = 30
window = np.ones(window_size)/float(window_size)
aapl_avg = np.convolve(aapl, window, 'same')

# create a new plot with a a datetime axis type
p = figure(width=800, height=350, x_axis_type="datetime")

# add renderers
p.circle(aapl_dates, aapl, size=4, color='darkgrey', alpha=0.2, legend='close')
p.line(aapl_dates, aapl_avg, color='red', legend='avg')

# NEW: customize by setting attributes
p.title.text = "AAPL One-Month Average"
p.legend.location = "top_left"
p.grid.grid_line_alpha=0
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Price'

p.ygrid.band_fill_color="gray"
p.ygrid.band_fill_alpha = 0.1

p.legend.click_policy="hide" # enable the click policy on the legend

# show the results
show(p)

## Categorical axes
To inform Bokeh that the `x-axis` is `categorical`, we pass this list of factors as the `x_range` argument to bokeh plotting figure:

```python
p = figure(x_range=fruits, ... )
```

Note that passing the list of factors is a convenient shorthand notation for creating a FactorRange. The equivalent explicit notation is:
```python
p = figure(x_range=FactorRange(field=fruits), ... )
```
This more explicit for is useful when you want to customize the FactorRange, e.g. by changing the range or category padding.

Next we can call vbar with the list of fruit name factors as the x coordinate, the bar height as the top coordinate, and optionally any width or other properties that we would like to set:
```python
p.vbar(x=fruits, top=[5, 3, 4, 2, 4, 6], width=0.9)
```

Now putting this stuff together we will see the output:

In [4]:
fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']

p = figure(x_range=fruits, plot_height=250, title="Fruit Counts",
           toolbar_location=None, tools="")

p.vbar(x=fruits, top=[5, 3, 4, 2, 4, 6], width=0.6)

p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

Often times we may want to have bars that are shaded some color. One way is to supply all the colors up front. This can be done by putting all the data, including the colors for each bar, in a `ColumnDataSource`. Then the name of the column containing the colors is passed to figure as the color (or line_color/fill_color) arguments. This is shown below:

In [5]:
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
counts = [5, 3, 4, 2, 4, 6]

source = ColumnDataSource(data=dict(fruits=fruits, counts=counts, color=Spectral6))

p = figure(x_range=fruits, y_range=(0,9), plot_height=250, title="Fruit Counts",
           toolbar_location=None, tools="")

p.vbar(x='fruits', top='counts', width=0.6, color='color', legend="fruits", source=source)

p.xgrid.grid_line_color = None
p.legend.orientation = "horizontal"
p.legend.location = "top_center"

show(p)

### Grouped

When creating bar charts, it is often desirable to visually display the data according to sub-groups. There are two basic methods that can be used, depending on your use case: using nested categorical coordinates, or applying vidual dodges.

### Nested Categories

If the coordinates of a plot range and data have two or three levels, then Bokeh will automatically group the factors on the axis, including a hierarchical tick labeling with separators between the groups. In the case of bar charts, this results in bars grouped together by the top-level factors. This is probably the most common way to achieve grouped bars, especially if you are starting from “tidy” data.

The example below shows this approach by creating a single column of coordinates that are each 2-tuples of the form (fruit, year). Accordingly, the plot groups the axes by fruit type, with a single call to vbar:

In [6]:
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ['2015', '2016', '2017']

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

# this creates [ ("Apples", "2015"), ("Apples", "2016"), ("Apples", "2017"), ("Pears", "2015), ... ]
x = [ (fruit, year) for fruit in fruits for year in years ]
counts = sum(zip(data['2015'], data['2016'], data['2017']), ()) # like an hstack in numpy

source = ColumnDataSource(data=dict(x=x, counts=counts))

p = figure(x_range=FactorRange(*x), plot_height=250, title="Fruit Counts by Year",
           toolbar_location=None, tools="")

p.vbar(x='x', top='counts', width=0.9, source=source)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

We can also apply a color mapping, similar to the earlier example. To obtain same grouped bar plot of fruits data as above, except with the bars shaded by the year, changethe vbar function call to use `factor_cmap` for the fill_color:

In [7]:
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral10

# use the palette to colormap based on the the x[1:2] values
p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white",
       fill_color=factor_cmap('x', palette=Spectral10, factors=years, start=1, end=2))
show(p)

Another method for achieving grouped bars is to explicitly specify a visual displacement for the bars. Such a visual offset is also referred to as a dodge.

In this scenario, our data is not “tidy”. Instead a single table with rows indexed by factors (fruit, year), we have separate series for each year. We can plot all the year series using separate calls to vbar but since every bar in each group has the same fruit factor, the bars would overlap visually. We can prevent this overlap and distinguish the bars visually by using the dodge() function to provide an offset for each different call to vbar:

In [8]:
from bokeh.core.properties import value
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ['2015', '2016', '2017']

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

source = ColumnDataSource(data=data)

p = figure(x_range=fruits, y_range=(0, 10), plot_height=250, title="Fruit Counts by Year",
           toolbar_location=None, tools="")

p.vbar(x=dodge('fruits', -0.25, range=p.x_range), top='2015', width=0.2, source=source,
       color="#c9d9d3", legend=value("2015"))

p.vbar(x=dodge('fruits',  0.0,  range=p.x_range), top='2016', width=0.2, source=source,
       color="#718dbf", legend=value("2016"))

p.vbar(x=dodge('fruits',  0.25, range=p.x_range), top='2017', width=0.2, source=source,
       color="#e84d60", legend=value("2017"))

p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)

Another common operation or bar charts is to stack bars on top of one another. Bokeh makes this easy to do with the specialized hbar_stack() and vbar_stack() functions. The example below shows the fruits data from above, but with the bars for each fruit type stacked instead of grouped:

In [9]:
fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ["2015", "2016", "2017"]
colors = ["#34A212", "#D3F543", "#98F5FF"]

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 4, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

source = ColumnDataSource(data=data)

p = figure(x_range=fruits, plot_height=250, title="Fruit Counts by Year",
           toolbar_location=None, tools="")

p.vbar_stack(years, x='fruits', width=0.9, color=colors, source=source,
             legend=[value(x) for x in years])

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)

Sometimes we may want to stack bars that have both positive and negative extents. The example below shows how it is possible to create such a stacked bar chart that is split by positive and negative values:

In [10]:
from bokeh.palettes import GnBu3, OrRd3

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ["2015", "2016", "2017"]

exports = {'fruits' : fruits,
           '2015'   : [2, 1, 4, 3, 2, 4],
           '2016'   : [5, 3, 4, 2, 4, 6],
           '2017'   : [3, 2, 4, 4, 5, 3]}
imports = {'fruits' : fruits,
           '2015'   : [-1, 0, -1, -3, -2, -1],
           '2016'   : [-2, -1, -3, -1, -2, -2],
           '2017'   : [-1, -2, -1, 0, -2, -2]}

p = figure(y_range=fruits, plot_height=250, x_range=(-16, 16), title="Fruit import/export, by year",
           toolbar_location=None)

p.hbar_stack(years, y='fruits', height=0.9, color=GnBu3, source=ColumnDataSource(exports),
             legend=["%s exports" % x for x in years])

p.hbar_stack(years, y='fruits', height=0.9, color=OrRd3, source=ColumnDataSource(imports),
             legend=["%s imports" % x for x in years])

p.y_range.range_padding = 0.1
p.ygrid.grid_line_color = None
p.legend.location = "top_left"
p.axis.minor_tick_line_color = None
p.outline_line_color = None

show(p)

## Gridplots and linked charts
In this notebook i will figure out some layout options for `bokeh` charts. The `gridplot()` function can be used to arrange Bokeh Plots in grid layout. Linking together various aspects of different plots can be a useful technique for data visualization. In Bokeh, such linkages are typically accomplished by sharing some plot component between plots.

### Gridplots
gridplot() allows to create a N x M layout for visualise several plots in an ordered way. It also collects all tools into a single toolbar, and the currently active tool is the same for all plots in the grid. **It is possible to leave “empty” spaces in the grid by passing None instead of a plot object.**

In [11]:
from bokeh.layouts import gridplot
from bokeh.palettes import Viridis3
from bokeh.plotting import figure

In [12]:
N = 100
x = np.linspace(0, 4*np.pi, N)
y0 = np.sin(x)
y1 = np.cos(x)
y2 = np.sin(x) + np.cos(x)

# create three plots
p1 = figure(plot_width=250, plot_height=250, title=None)
p1.circle(x, y0, size=3, color=Viridis3[0])

p2 = figure(plot_width=250, plot_height=250, title=None)
p2.circle(x, y1, size=3, color=Viridis3[1])

p3 = figure(plot_width=250, plot_height=250, title=None)
p3.circle(x, y2, size=3, color=Viridis3[2])

# make a grid
grid = gridplot([[p1, p2], [None, p3]])

# show the results
show(grid)

For convenience you can also just pass a list of plots, and specify the number of columns you want in your grid. For example,

`gridplot([[s1, s2], [s3, None]])`

and

`gridplot([s1, s2, s3], ncols=2)`

Furthermore, you can specify the size of the plots in the gridplot istruction.

In [13]:
from bokeh.io import output_file, show
from bokeh.layouts import gridplot
from bokeh.palettes import Viridis3
from bokeh.plotting import figure

N = 100
x = np.linspace(0, 4*np.pi, N)
y0 = np.sin(x)
y1 = np.cos(x)
y2 = np.sin(x) + np.cos(x)

# create three plots
s1 = figure()
s1.circle(x, y0, size=3, color=Viridis3[0])
s2 = figure()
s2.circle(x, y1, size=3, color=Viridis3[1])
s3 = figure()
s3.circle(x, y2, size=3, color=Viridis3[2])

# make a grid
grid = gridplot([s1, s2, s3], ncols=2, plot_width=250, plot_height=250)

# show the results
show(grid)

### Linked panning and brushing

Linking together various aspects of different plots can be a useful technique for data visualization. In Bokeh, such linkages are typically accomplished by sharing some plot component between plots. Below is an example that demonstrates linked panning (where changing the range of one plot causes others to update) by sharing range objects between the plots. Some other things to look out for in this example:

- calling `figure()` multiple times to create multiple plots
- using `gridplot()` to arrange several plots in an array
- showing new glyphs using new glyph methods Figure.triangle and Figure.square
- hiding the toolbar by setting toolbar_location to None
- setting convenience arguments color (sets both line_color and fill_color) and alpha (sets both line_alpha and fill_alpha)


In [14]:
# prepare some data
N = 100
x = np.linspace(0, 4*np.pi, N)
y0 = np.sin(x)
y1 = np.cos(x)
y2 = np.sin(x) + np.cos(x)

# create a new plot
s1 = figure(title=None)
s1.circle(x, y0, size=3, color="navy", alpha=0.5)

# NEW: create a new plot and share both ranges
s2 = figure(x_range=s1.x_range, y_range=s1.y_range, title=None)
s2.circle(x, y1, size=3, color="firebrick", alpha=0.5)

# NEW: create a new plot and share only one range
s3 = figure(x_range=s1.x_range, title=None)
s3.circle(x, y2, size=3, color="olive", alpha=0.5)

# NEW: put the subplots in a gridplot
p = gridplot([[s1, s2, s3]], toolbar_location=None, plot_width=250, plot_height=250)

# show the results
show(p)

Another linkage that is often useful is linked brushing (where a selection on one plot causes a selection to update on other plots). Below is an example that demonstrates linked brushing by sharing a [ColumnDataSource](https://bokeh.pydata.org/en/latest/docs/reference/models/sources.html#bokeh.models.sources.ColumnDataSource) between two plots:

In [15]:
from bokeh.models import ColumnDataSource
# prepare some date
N = 300
x = np.linspace(0, 4*np.pi, N)
y0 = np.sin(x)
y1 = np.cos(x)

# NEW: create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=350, height=350, title=None)
left.circle('x', 'y0', source=source)

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=350, height=350, title=None)
right.circle('x', 'y1', source=source)

# put the subplots in a gridplot
p = gridplot([[left, right]])

# show the results
show(p)

# Interaction with widgets and JS CallBacks

## Adding Widgets

Widgets are interactive controls that can be added to Bokeh applications to provide a front end user interface to a visualization. They can drive new computations, update plots, and connect to other programmatic functionality.
Widgets can be added directly to the document root or nested inside a layout. There are two ways to program a widget’s functionality:

- Use the `CustomJS` callback. This will work in standalone HTML documents.
- Use `bokeh serve` to start the Bokeh server and set up event handlers. *[DESCRIBED IN OTHER NOTEBOOKS]*

### Buttons

In [16]:
from bokeh.layouts import widgetbox

In [17]:
from bokeh.models.widgets import Button

button_warning = Button(label="Foo Warning", button_type="warning")
button_success = Button(label="Foo Success", button_type="success")

show(widgetbox(button_warning, button_success))

### Checkbox button group

In [18]:
from bokeh.models.widgets import CheckboxButtonGroup

checkbox_button_group = CheckboxButtonGroup(
        labels=["Option 1", "Option 2", "Option 3"], 
        active=[1, 2]) # active param show selected the slice you specify

show(widgetbox(checkbox_button_group))

### Checkbox group

In [19]:
from bokeh.models.widgets import CheckboxGroup

checkbox_group = CheckboxGroup(
        labels=["Option 1", "Option 2", "Option 3"], active=[0, 1])

show(widgetbox(checkbox_group))

## Data Table
Bokeh provides a sophisticated data table widget based on SlickGrid. Note that since the table is configured with a data source object, any plots that share this data source will automatically have selections linked between the plot and the table (even in static HTML documents). 

*The data table is sortable by clicking on the column name*

In [20]:
from datetime import date
from random import randint
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn

data = dict(
        dates=[date(2017, 3, i+1) for i in range(10)],
        downloads=[randint(0, 100) for i in range(10)],
    )
source = ColumnDataSource(data)

columns = [
        TableColumn(field="dates", title="Date", formatter=DateFormatter()),
        TableColumn(field="downloads", title="Downloads"),
    ]
data_table = DataTable(source=source, columns=columns, width=400, height=280)

show(widgetbox(data_table))

## Sliders

In [21]:
from bokeh.models.widgets import Slider, RangeSlider

slider = Slider(start=0, end=10, value=3, step=.1, title="Slider")
range_slider = RangeSlider(start=0, end=10, value=(5,9), step=.1, title="Range Slider")

show(widgetbox(slider, range_slider))

## Tab panes

In [22]:
from bokeh.models.widgets import Panel, Tabs
from bokeh.plotting import figure

p1 = figure(plot_width=300, plot_height=300)
p1.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=20, color="red", alpha=0.5)
tab1 = Panel(child=p1, title="circle")

p2 = figure(plot_width=300, plot_height=300)
p2.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], line_width=3, color="blue", alpha=0.5)
tab2 = Panel(child=p2, title="line")

tabs = Tabs(tabs=[ tab1, tab2 ])

show(tabs)

## JavaScript CallBacks
While the main goal of Bokeh is to provide a path to create rich interactive visualizations in the browser, purely from python, there will always be specialized use-cases that are outside the capabilities of the core library. For this reason, Bokeh provides different ways for users to supply custom JavaScript when necessary, so that users may add custom or specialized behaviours in response to property changes and other events.
To supply a snippet of JavaScript code that should be executed (in the browser) when some event occurs, use the `CustomJS model`:
```python
from bokeh.models.callbacks import CustomJS

callback = CustomJS(args=dict(xr=plot.x_range), code="""

// JavaScript code goes here

var a = 10;

// the model that triggered the callback is cb_obj:
var b = cb_obj.value;

// models passed as args are automagically available
xr.start = a;
xr.end = b;

""")
```

### CustomJS for Model Property Events¶
These CustomJS callbacks can be attached to property change events on any Bokeh model, using the `js_on_change` method of Bokeh models:

In [23]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import figure

x = [x*0.005 for x in range(0, 200)]
y = x

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var f = cb_obj.value
    x = data['x']
    y = data['y']
    for (i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    source.change.emit();
""")

slider = Slider(start=0.1, end=4, value=1, step=.1, title="power")
slider.js_on_change('value', callback)

layout = column(slider, plot)

show(layout)

The above example shows how to attach a CustomJS callback to a Slider widget, so that whenever the slider value updates, the callback is executed to update some data.

### CustomJS with a Python function
A CustomJS callback can also be implemented as a Python function, which is then translated to JavaScript using PyScript. This makes it easier for users to define client-side interactions without having to learn JavaScript. 

To use this functionality you need the Flexx library (install with `conda install -c bokeh flexx` or `pip install flexx`).

In [24]:
!pip install pscript

In [25]:
## NOTE: This requires flexx or pscript installed: conda install -c conda-forge pscript

from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import figure

x = [x*0.05 for x in range(0, 200)]
y = x

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

def callback(source=source, window=None):
    data = source.data
    f = cb_obj.value
    x, y = data['x'], data['y']
    for i in range(len(x)):
        y[i] = window.Math.pow(x[i], f)
    source.change.emit()

slider = Slider(start=0.1, end=4, value=1, step=.1, title="power",
                callback=CustomJS.from_py_func(callback))

layout = column(slider, plot)

show(layout)

### CustomJS for Selections
Bokeh also provides the means to specify the same kind of callback to be executed whenever a selection changes. As a simple demonstration, the example below simply copies selected points on the first plot to the second. However, more sophisticated actions and computations are easily constructed in a similar way.

In [26]:
from random import random
from bokeh.layouts import row
from bokeh.models import CustomJS, ColumnDataSource
from bokeh.plotting import figure

x = [random() for x in range(500)]
y = [random() for y in range(500)]

s1 = ColumnDataSource(data=dict(x=x, y=y))
p1 = figure(plot_width=400, plot_height=400, tools="lasso_select", title="Select Here")
p1.circle('x', 'y', source=s1, alpha=0.6)

s2 = ColumnDataSource(data=dict(x=[], y=[]))
p2 = figure(plot_width=400, plot_height=400, x_range=(0, 1), y_range=(0, 1),
            tools="", title="Watch Here")
p2.circle('x', 'y', source=s2, alpha=0.6)

s1.callback = CustomJS(args=dict(s2=s2), code="""
        var inds = cb_obj.selected['1d'].indices;
        var d1 = cb_obj.data;
        var d2 = s2.data;
        d2['x'] = []
        d2['y'] = []
        for (i = 0; i < inds.length; i++) {
            d2['x'].push(d1['x'][inds[i]])
            d2['y'].push(d1['y'][inds[i]])
        }
        s2.change.emit();
    """)

layout = row(p1, p2)

show(layout)